In [220]:
import numpy as np
import networkx as nx
from pyvis.network import Network
import math
import community as community_louvain

import seaborn as sns
import pandas as pd
import os

os.chdir(os.path.expanduser('~/Documents/vivarium-ecoli'))

import matplotlib.pyplot as plt
import dill
import xmltodict
import os
import requests
from bs4 import BeautifulSoup
from ecoli.processes.metabolism_redux import NetworkFlowModel, FlowResult, MetabolismRedux


FREE_RXNS = ["TRANS-RXN-145", "TRANS-RXN0-545", "TRANS-RXN0-474"]

In [221]:
# Change the environment name here
env_name = 'Nitrate_as_reduction_target'

In [222]:
condition_df = pd.read_excel("ecoli/experiments/new_environments_test/done.xlsx")
condition_df["Add"] = condition_df["Add"].apply(lambda x: set(str(x).split(", ") if x == x else set()))
condition_df["Remove"] = condition_df["Remove"].apply(lambda x: set(str(x).split(", ") if x == x else set()))

row_num = condition_df[condition_df['Environment'] == env_name].index[0]
important_add = condition_df["Add"].loc[row_num]
important_remove = condition_df["Remove"].loc[row_num]

In [223]:
time = '50'
date = '2023-07-21'
experiment = 'fba_new_environments'
entry = f'{experiment}_{env_name}_{time}_{date}'
folder = f'out/fba_new_env/{entry}/'

f = open(folder + 'agent_steps.pkl', 'rb')
agent = dill.load(f)
f.close()

metabolism = agent['ecoli-metabolism-redux']
stoichiometry = metabolism.stoichiometry

output = np.load(folder + 'output.npy',allow_pickle='TRUE').item()
output = output['agents']['0']
fba = output['listeners']['fba_results']
mass = output['listeners']['mass']
bulk = pd.DataFrame(output['bulk'])

sim_fluxes = pd.DataFrame(fba["estimated_fluxes"], columns = metabolism.reaction_names)
sim_fluxes = pd.DataFrame(sim_fluxes.loc[24, :].abs().sort_values(ascending=False))

metabolites_idx = {species: i for i, species in enumerate(metabolism.metabolite_names)}
index_to_met = {v: k for k, v in metabolites_idx.items()}

reaction_idx = {reaction: i for i, reaction in enumerate(metabolism.reaction_names)}
index_to_rxn = {v: k for k, v in reaction_idx.items()}

metabolism.catalyst_ids = metabolism.parameters['catalyst_ids']
catalyst_idx = {catalyst: i for i, catalyst in enumerate(metabolism.catalyst_ids)}

In [224]:
heatmap_df = pd.read_excel("out/new_env_analysis/heatmap.xlsx")

# Building the Graph

In [225]:
G = nx.DiGraph()

met_length = len(metabolism.metabolite_names)
reaction_length = np.shape(stoichiometry)[1]
met_not_include = ["ATP[c]", "WATER[c]", "PROTON[p]", "Pi[c]", "PROTON[c]", "ADP[c]", "Pi[p]", "WATER[p]"]
# met_not_include = []
do_not_include = []

for met in met_not_include:
    do_not_include.append(metabolites_idx[met])

def plot_metabolite(G, metabolite, heatmap_df, mode):
    index = metabolites_idx[metabolite]
    to_return = []
    for k in range(reaction_length):
        if stoichiometry[index, k] < 0:
            reaction = index_to_rxn[k]
            clean_reaction = clean_reaction_name(reaction)

            row_num = heatmap_df[heatmap_df['Reactions'] == reaction].index[0]
            if heatmap_df[env_name].loc[row_num] == 0:
                continue
            else:
                flux = math.log(heatmap_df[env_name].loc[row_num], 10)

            if flux < 0 and mode == 'green':
                continue
            elif flux > 0 and mode == 'red':
                continue

            if flux > 0:
                color = 'green'
            else:
                color = 'red'

            G.add_edge(metabolite, clean_reaction, color=color, title=flux)
            for l in range(met_length):
                if stoichiometry[l, k] > 0:
                    if l != index and l not in do_not_include:
                        product = index_to_met[l]
                        to_return.append(product)
                        G.add_edge(clean_reaction, product, color=color, title=flux)
                elif stoichiometry[l, k] < 0:
                    if l != index and l not in do_not_include:
                        reactant = index_to_met[l]
                        G.add_edge(reactant, clean_reaction, color=color, title=flux)

    return to_return

In [226]:
def clean_reaction_name(reaction):
    if '/' in reaction:
        index = reaction.find('RXN')
        if (reaction[index+3] == '-' and reaction[index+4].isdigit()) or (reaction[index+4] == '-' and reaction[index+5].isdigit()):
            index = index + 4
            if reaction[index] == '-':
                index += 1
            while reaction[index].isdigit():
                index += 1
            reaction = reaction[:index]
        else:
            reaction = reaction[:index+3]
    elif '__' in reaction:
        reaction = reaction[:reaction.find('__')]
    elif '(reverse)' in reaction:
        reaction = reaction[:reaction.find('(reverse)') - 1]
    elif '[' in reaction:
        reaction = reaction[:reaction.find('[') - 1]
    elif ' ' in reaction:
        reaction = reaction[:reaction.find('[')]
    return reaction

In [227]:
# Change Parameters Here
k = 10
mode = 'green'

In [228]:
if mode == 'red':
    return_met_1 = important_remove
else:
    return_met_1 = important_add
heatmap_df.sort_values(by=[env_name], ascending=False, inplace=True)
return_met_2 = []

for i in range(k):
    for met in return_met_1:
        if met is None:
            continue
        return_met_2 += plot_metabolite(G, met, heatmap_df, mode)
    return_met_2 = list(set(return_met_2))
    return_met_1 = return_met_2
    return_met_2 = []

In [229]:
closeness = nx.closeness_centrality(G)
sort_closeness = sorted(closeness.items(), key=lambda x:x[1], reverse=True)

sort_closeness = sort_closeness[:8]
nt = Network('1000px', '1500px', notebook=True, cdn_resources='in_line', directed=True)
nt.from_nx(G)

for tup in sort_closeness:
    nt.get_node(tup[0])['color'] = 'blue'

for n in nt.nodes:
    if n['label'] in metabolites_idx:
        n['shape'] = 'star'
    if n['label'] in important_add:
        n['color'] = 'orange'
    if n['label'] in important_remove:
        n['color'] = 'purple'


# nt.toggle_physics(True)
nt.show_buttons(filter_=['physics'])
nt.show(f'out/new_env_analysis/met_visualization/{mode}/{env_name}_metabolite_{k}{mode}_graph.html')

out/new_env_analysis/met_visualization/green/Nitrate_as_reduction_target_metabolite_10green_graph.html
